# An sample to finetune vicuna on SageMaker

In [ ]:
## Update sagemaker python sdk version
!pip install -U sagemaker

In [23]:
import sagemaker
import boto3
from sagemaker import get_execution_role

sess = sagemaker.Session()
role = get_execution_role()
sagemaker_default_bucket = sess.default_bucket()

account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [76]:
## download training script from github
!rm -rf ./FastChat
!git clone https://github.com/lm-sys/FastChat.git
!cp ./s5cmd ./FastChat/

Cloning into 'FastChat'...
remote: Enumerating objects: 3587, done.
remote: Counting objects: 100% (1627/1627), done.
remote: Compressing objects: 100% (461/461), done.
remote: Total 3587 (delta 1431), reused 1222 (delta 1164), pack-reused 1960
Receiving objects: 100% (3587/3587), 30.06 MiB | 38.24 MiB/s, done.
Resolving deltas: 100% (2519/2519), done.


## Download pretrained model from HuggingFace Hub

To avoid download model from Huggingface hub failure, we download first and push those model files to S3 bucket first.

In [3]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [81]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_cache_path = Path("./model")
local_cache_path.mkdir(exist_ok=True)

model_name = "pinkmanlove/llama-7b-hf"#decapoda-research/llama-13b-hf

# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.pt", "*.bin", "*.model"]

model_download_path = snapshot_download(
    repo_id=model_name,
    cache_dir=local_cache_path,
    #allow_patterns=allow_patterns,
)

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

**Upload model files to S3**

In [6]:
# Get the model files path
import os
from glob import glob

local_model_path = None

paths = os.walk(r'./model')
for root, dirs, files in paths:
    for file in files:
        if file == 'config.json':
            print(os.path.join(root,file))
            local_model_path = str(os.path.join(root,file))[0:-11]
            print(local_model_path)
if local_model_path == None:
    print("Model download may failed, please check prior step!")

./model/models--pinkmanlove--llama-7b-hf/snapshots/b3cde76468bad3c085ead29707ee7481121a4ca0/config.json
./model/models--pinkmanlove--llama-7b-hf/snapshots/b3cde76468bad3c085ead29707ee7481121a4ca0/


In [98]:
%%script env sagemaker_default_bucket=$sagemaker_default_bucket local_model_path=$local_model_path bash

chmod +x ./s5cmd
./s5cmd sync ${local_model_path} s3://${sagemaker_default_bucket}/llama/pretrain/pinkmanlove/llama-7b-hf/ 

rm -rf model

Name:
  sync - sync objects

Usage:
  sync [options] source destination

Options:
  --delete                       delete objects in destination but not in source (default: false)
  --size-only                    make size of object only criteria to decide whether an object should be synced (default: false)
  --no-follow-symlinks           do not follow symbolic links (default: false)
  --storage-class value          set storage class for target ('STANDARD','REDUCED_REDUNDANCY','GLACIER','STANDARD_IA','ONEZONE_IA','INTELLIGENT_TIERING','DEEP_ARCHIVE')
  --concurrency value, -c value  number of concurrent parts transferred between host and remote server (default: 5)
  --part-size value, -p value    size of each part transferred between host and remote server, in MiB (default: 50)
  --sse value                    perform server side encryption of the data at its destination, e.g. aws:kms
  --sse-kms-key-id value         customer master key (CMK) id for SSE-KMS encryption; leave it out if

ERROR "sync $local_model_path s3://$sagemaker_default_bucket/llama/pretrain/pinkmanlove/llama-7b-hf/": given object not found


## Prepare docker image

In [46]:
%%writefile Dockerfile
## You should change below region code to the region you used, here sample is use us-west-2
From 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-training:1.13.1-transformers4.26.0-gpu-py39-cu117-ubuntu20.04 
#From pytorch/pytorch:1.5-cuda10.1-cudnn7-runtime

ENV LANG=C.UTF-8
ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE

# RUN python3 -m pip install git+https://github.com/huggingface/transformers.git@97a3d16a6941294d7d76d24f36f26617d224278e

RUN pip3 uninstall -y deepspeed && pip3 install deepspeed
RUN python3 -m pip install transformers==4.28.0


## Make all local GPUs visible
ENV NVIDIA_VISIBLE_DEVICES="all"

Overwriting Dockerfile


In [47]:
## You should change below region code to the region you used, here sample is use us-west-2
!aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-west-2.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


**Build image and push to ECR.**

In [48]:
## define repo name, should contain *sagemaker* in the name
repo_name = "sagemaker-vicuna-demo"

In [49]:
%%script env repo_name=$repo_name bash

#!/usr/bin/env bash

# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.
# The name of our algorithm
algorithm_name=${repo_name}

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon  68.01GB
Step 1/7 : From 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-training:1.13.1-transformers4.26.0-gpu-py39-cu117-ubuntu20.04
 ---> c5a6ef695006
Step 2/7 : ENV LANG=C.UTF-8
 ---> Using cache
 ---> af49cfa7feae
Step 3/7 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 287106637dc6
Step 4/7 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 773b4cf30c90
Step 5/7 : RUN pip3 uninstall -y deepspeed && pip3 install deepspeed
 ---> Using cache
 ---> ce72201e73cd
Step 6/7 : RUN python3 -m pip install transformers==4.28.0
 ---> Using cache
 ---> e234794bbe5c
Step 7/7 : ENV NVIDIA_VISIBLE_DEVICES="all"
 ---> Using cache
 ---> bb43a66885f0
Successfully built bb43a66885f0
Successfully tagged sagemaker-vicuna-demo:latest
The push refers to repository [687912291502.dkr.ecr.us-west-2.amazonaws.com/sagemaker-vicuna-demo]
1e9d9d5ddefd: Preparing
02a87473f68b: Preparing
f8dae5c3df1e: Preparing
e3221f18601a: Prepa

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



### Generate the deepspeed config

In [50]:
%%writefile ds.json
{
  "fp16": {
    "enabled": true,
    "auto_cast": false,
    "loss_scale": 0,
    "initial_scale_power": 16,
    "loss_scale_window": 1000,
    "hysteresis": 2,
    "min_loss_scale": 1
  },
  "optimizer": {
    "type": "AdamW",
    "params": {
      "lr": "auto",
      "betas": "auto",
      "eps": "auto",
      "weight_decay": "auto"
    }
  },
  "scheduler": {
    "type": "WarmupLR",
    "params": {
      "warmup_min_lr": "auto",
      "warmup_max_lr": "auto",
      "warmup_num_steps": "auto"
    }
  },
  "zero_optimization": {
    "stage": 3,
    "overlap_comm": true,
    "contiguous_gradients": true,
    "sub_group_size": 1e9,
    "reduce_bucket_size": "auto",
    "stage3_prefetch_bucket_size": "auto",
    "stage3_param_persistence_threshold": "auto",
    "stage3_max_live_parameters": 1e9,
    "stage3_max_reuse_distance": 1e9,
    "stage3_gather_16bit_weights_on_model_save": true
  },
  "gradient_accumulation_steps": "auto",
  "gradient_clipping": "auto",
  "steps_per_print": 2000,
  "train_batch_size": "auto",
  "train_micro_batch_size_per_gpu": "auto",
  "wall_clock_breakdown": false
}

Overwriting ds.json


**Generate training entrypoint script.**

**Note: DO NOT CHANGE BELOW VAlUE OF "output_dir" and "cache_dir", keep it "/tmp/llama_out" and "/tmp".**

Below is just a testing to fine-tune on a sample dataset (just 8 samples), you could change ```data_path``` to your dataset for furthur fine tune.

For the dataset download, you could follow the way how to download pretrain model:
```
./s5cmd sync s3://$MODEL_S3_BUCKET/llama/pretrain/7B/* /tmp/llama_pretrain/
```

It is recommend to use the folder ```/tmp/dataset/```.

## Notice

We modified some parts of ```FastChat/fastchat/train/train.py```, such as how to save model.

In [51]:
!mv FastChat/fastchat/train/train.py FastChat/fastchat/train/train_bak.py

In [99]:
%%writefile FastChat/fastchat/train/train.py
# Adopted from tatsu-lab@stanford_alpaca. Below is the original copyright:
#    Copyright 2023 Rohan Taori, Ishaan Gulrajani, Tianyi Zhang, Yann Dubois, Xuechen Li
#
#    Licensed under the Apache License, Version 2.0 (the "License");
#    you may not use this file except in compliance with the License.
#    You may obtain a copy of the License at
#
#        http://www.apache.org/licenses/LICENSE-2.0
#
#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.

import copy
from dataclasses import dataclass, field
import json
import pathlib
from typing import Dict, Optional, Sequence

import torch
from torch.utils.data import Dataset
import transformers
from transformers import Trainer
####
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, AutoTokenizer
from transformers.models.llama.tokenization_llama import LlamaTokenizer
####
from transformers.trainer_pt_utils import LabelSmoother

from fastchat.conversation import get_conv_template, SeparatorStyle

IGNORE_TOKEN_ID = LabelSmoother.ignore_index


@dataclass
class ModelArguments:
    model_name_or_path: Optional[str] = field(default="facebook/opt-125m")


@dataclass
class DataArguments:
    data_path: str = field(default=None,
                           metadata={"help": "Path to the training data."})
    lazy_preprocess: bool = False


@dataclass
class TrainingArguments(transformers.TrainingArguments):
    cache_dir: Optional[str] = field(default=None)
    optim: str = field(default="adamw_torch")
    model_max_length: int = field(
        default=512,
        metadata={
            "help":
            "Maximum sequence length. Sequences will be right padded (and possibly truncated)."
        },
    )


local_rank = None


def rank0_print(*args):
    if local_rank == 0:
        print(*args)


def safe_save_model_for_hf_trainer(trainer: transformers.Trainer,
                                   output_dir: str):
    """Collects the state dict and dump to disk."""
    state_dict = trainer.model.state_dict()
    if trainer.args.should_save:
        cpu_state_dict = {
            key: value.cpu()
            for key, value in state_dict.items()
        }
        del state_dict
        trainer._save(output_dir, state_dict=cpu_state_dict)  # noqa


def preprocess(
    sources,
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    conv = get_conv_template("vicuna_v1.1").copy()
    roles = {"human": conv.roles[0], "gpt": conv.roles[1]}

    # Apply prompt templates
    conversations = []
    for i, source in enumerate(sources):
        if roles[source[0]["from"]] != conv.roles[0]:
            # Skip the first one if it is not from human
            source = source[1:]

        conv.messages = []
        for j, sentence in enumerate(source):
            role = roles[sentence["from"]]
            assert role == conv.roles[j % 2], f"{i}"
            conv.append_message(role, sentence["value"])
        conversations.append(conv.get_prompt())

    # Tokenize conversations
    input_ids = tokenizer(
        conversations,
        return_tensors="pt",
        padding="max_length",
        max_length=tokenizer.model_max_length,
        truncation=True,
    ).input_ids
    targets = input_ids.clone()

    assert conv.sep_style == SeparatorStyle.ADD_COLON_TWO

    # Mask targets
    sep = conv.sep + conv.roles[1] + ": "
    for conversation, target in zip(conversations, targets):
        total_len = int(target.ne(tokenizer.pad_token_id).sum())

        rounds = conversation.split(conv.sep2)
        cur_len = 1
        for i, rou in enumerate(rounds):
            if rou == "":
                break

            parts = rou.split(sep)
            if len(parts) != 2:
                break
            parts[0] += sep
            round_len = len(tokenizer(rou).input_ids)
            instruction_len = len(tokenizer(parts[0]).input_ids) - 2

            target[cur_len:cur_len+instruction_len] = (
                IGNORE_TOKEN_ID)

            #rank0_print(tokenizer.decode(target[cur_len+instruction_len:cur_len+round_len]))

            cur_len += round_len
        target[cur_len:] = IGNORE_TOKEN_ID

        if cur_len < tokenizer.model_max_length:
            if cur_len != total_len:
                rank0_print(f"WARNING: tokenization mismatch "
                            f"{cur_len} vs. {total_len}")

    return dict(input_ids=input_ids, labels=targets,
                attention_mask=input_ids.ne(tokenizer.pad_token_id))


class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, data_path: str,
                 tokenizer: transformers.PreTrainedTokenizer):
        super(SupervisedDataset, self).__init__()
        rank0_print("Loading data...")
        list_data_dict = json.load(open(data_path, "r"))

        rank0_print("Formatting inputs...")
        sources = [example["conversations"] for example in list_data_dict]
        data_dict = preprocess(sources, tokenizer)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        self.attention_mask = data_dict["attention_mask"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i],
                    labels=self.labels[i],
                    attention_mask=self.attention_mask[i])


class LazySupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, data_path: str,
                 tokenizer: transformers.PreTrainedTokenizer):
        super(LazySupervisedDataset, self).__init__()
        self.tokenizer = tokenizer

        rank0_print("Loading data...")
        list_data_dict = json.load(open(data_path, "r"))

        rank0_print("Formatting inputs...Skip in lazy mode")
        self.tokenizer = tokenizer
        self.list_data_dict = list_data_dict

    def __len__(self):
        return len(self.list_data_dict)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        sources = self.list_data_dict[i]
        if isinstance(i, int):
            sources = [sources]
        data_dict = preprocess([e["conversations"] for e in sources],
            self.tokenizer)
        if isinstance(i, int):
            data_dict = dict(input_ids=data_dict["input_ids"][0],
                             labels=data_dict["labels"][0],
                             attention_mask=data_dict["attention_mask"][0])
        return data_dict


def make_supervised_data_module(tokenizer: transformers.PreTrainedTokenizer,
                                data_args) -> Dict:
    """Make dataset and collator for supervised fine-tuning."""
    dataset_cls = (LazySupervisedDataset
                   if data_args.lazy_preprocess else SupervisedDataset)
    train_dataset = dataset_cls(tokenizer=tokenizer,
                                data_path=data_args.data_path)
    return dict(train_dataset=train_dataset,
                eval_dataset=None)


def train():
    global local_rank

    parser = transformers.HfArgumentParser(
        (ModelArguments, DataArguments, TrainingArguments))
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()
    local_rank = training_args.local_rank
    model = transformers.AutoModelForCausalLM.from_pretrained(
        model_args.model_name_or_path,
        cache_dir=training_args.cache_dir,
    )
    tokenizer = LlamaTokenizer.from_pretrained( #transformers.AutoTokenizer
        model_args.model_name_or_path,
        cache_dir=training_args.cache_dir,
        model_max_length=training_args.model_max_length,
        padding_side="right",
        use_fast=False,
    )
#####
#     tokenizer.pad_token = tokenizer.unk_token
    if tokenizer.pad_token is None:
        print("-----------no pad token and add special token PAD----")
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        model.resize_token_embeddings(len(tokenizer))
######
    data_module = make_supervised_data_module(tokenizer=tokenizer,
                                              data_args=data_args)
    trainer = Trainer(model=model,
                      tokenizer=tokenizer,
                      args=training_args,
                      **data_module)

    if list(pathlib.Path(training_args.output_dir).glob("checkpoint-*")):
        trainer.train(resume_from_checkpoint=True)
    else:
        trainer.train()
#     trainer.save_state()
#     safe_save_model_for_hf_trainer(trainer=trainer,
#                                    output_dir=training_args.output_dir)


    tokenizer.save_pretrained(training_args.output_dir)
    trainer.save_model(training_args.output_dir)


if __name__ == "__main__":
    train()

Overwriting FastChat/fastchat/train/train.py


Here we use sample dataset - sharegpt_test.json for testing.

### 单机多卡 deepspeed

In [119]:
%%writefile ./FastChat/ds-train.sh
#!/bin/bash

chmod +x ./s5cmd
./s5cmd sync s3://$MODEL_S3_BUCKET/llama/pretrain/pinkmanlove/llama-7b-hf/* /tmp/llama_pretrain/

#cd FastChat && pip install -e . && cd ..
pip install -e .

deepspeed --num_gpus=8 ./fastchat/train/train_mem.py \
    --deepspeed ds.json \
    --model_name_or_path "/tmp/llama_pretrain/" \
    --data_path data/dummy_conversation.json \
    --output_dir "/tmp/llama_out" \
    --num_train_epochs 1 \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size  2 \
    --gradient_accumulation_steps 8 \
    --evaluation_strategy "no" \
    --save_strategy "no" \
    --save_steps 2000 \
    --save_total_limit 1 \
    --learning_rate 2e-5 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --cache_dir '/tmp' \
    --model_max_length 2048 \
    --gradient_checkpointing True \
    --lazy_preprocess True \
    --fp16 True \
    --tf32 True \
    --report_to "none"

if [ $? -eq 1 ]; then
    echo "Training script error, please check CloudWatch logs"
    exit 1
fi

./s5cmd sync /tmp/llama_out s3://$MODEL_S3_BUCKET/llama/output/$(date +%Y-%m-%d-%H-%M-%S)/

Overwriting ./FastChat/ds-train.sh


In [120]:
## The image uri which is build and pushed above
image_uri = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account, region, repo_name)
image_uri

'687912291502.dkr.ecr.us-west-2.amazonaws.com/sagemaker-vicuna-demo:latest'

In [121]:
## set train_data_path to your training dataset path in s3
train_data_path = f's3://{sagemaker_default_bucket}/llama/train_data/'

inputs = {'train': train_data_path}

In [ ]:
import time
from sagemaker.estimator import Estimator

environment = {
              'MODEL_S3_BUCKET': sagemaker_default_bucket # The bucket to store pretrained model and fine-tune model
}

base_job_name = 'vicuna-demo'         

instance_type = 'ml.p4d.24xlarge'

estimator = Estimator(role=role,
                      entry_point='ds-train.sh',
                      source_dir='./FastChat/',
                      base_job_name=base_job_name,
                      instance_count=1,
                      instance_type=instance_type,
                      image_uri=image_uri,
                      environment=environment,
                      disable_profiler=True,
                      debugger_hook_config=False,
                      max_run=24*60*60*2)

estimator.fit()
#estimator.fit(inputs)

### 多机多卡 torch distribute + deepspeed 

In [131]:
%%writefile ./FastChat/ds-train-distribute.sh
#!/bin/bash
SM_MASTER="${SM_MASTER}"
SM_MASTER_ADDR="${SM_MASTER_ADDR}"
CURRENT_HOST="${SM_CURRENT_HOST}"


IFS=',' read -ra hosts_array <<< "${SM_HOSTS}"
NNODES=${#hosts_array[@]}
NODE_RANK=0

for i in "${!hosts_array[@]}"; do
    if [[ "${hosts_array[$i]}" == *${CURRENT_HOST}* ]]; then
        echo "host index：$i"
        NODE_RANK="$i" 
    fi
done
   
    
MASTER_PORT="23456"
export NCCL_SOCKET_IFNAME="eth0"

#Configure the distributed arguments for torch.distributed.launch.
GPUS_PER_NODE="$SM_NUM_GPUS"
DISTRIBUTED_ARGS="--nproc_per_node $GPUS_PER_NODE \
                  --nnodes $NNODES --node_rank $NODE_RANK \
                  --master_addr $MASTER_ADDR \
                  --master_port $MASTER_PORT"


SAVE_PATH="${SM_WORKING_DIR}/results"
LOG_FILE="${SAVE_PATH}/log.txt"


chmod +x ./s5cmd
./s5cmd sync s3://$MODEL_S3_BUCKET/llama/pretrain/pinkmanlove/llama-7b-hf/* /tmp/llama_pretrain/

#cd FastChat && pip install -e . && cd ..
pip install -e .


DEEPSPEED_OPTS="""
  ./fastchat/train/train_mem.py 
    --deepspeed ds.json 
    --model_name_or_path "/tmp/llama_pretrain/" 
    --data_path data/dummy_conversation.json 
    --output_dir "/tmp/llama_out" 
    --num_train_epochs 1 
    --per_device_train_batch_size 1 
    --per_device_eval_batch_size  1 
    --gradient_accumulation_steps 4 
    --evaluation_strategy "no" 
    --save_strategy "no" 
    --save_steps 2000 
    --save_total_limit 1 
    --learning_rate 2e-5 
    --weight_decay 0. 
    --warmup_ratio 0.03 
    --lr_scheduler_type "cosine" 
    --logging_steps 1 
    --cache_dir '/tmp' 
    --model_max_length 2048 
    --gradient_checkpointing True 
    --lazy_preprocess True 
    --fp16 True 
    --tf32 True 
    --report_to "none"
"""    

CMD="torchrun ${DISTRIBUTED_ARGS} ${DEEPSPEED_OPTS}"
echo ${CMD}
${CMD} 2>&1 
echo "begin to upload trained model"
if [[ "${CURRENT_HOST}" == "${SM_MASTER}" ]]; then  
    ./s5cmd sync /tmp/llama_out s3://$MODEL_S3_BUCKET/llama/output/$(date +%Y-%m-%d-%H-%M-%S)/
fi



Overwriting ./FastChat/ds-train-distribute.sh


In [133]:
import time
from sagemaker.estimator import Estimator

environment = {
              'MODEL_S3_BUCKET': sagemaker_default_bucket # The bucket to store pretrained model and fine-tune model
}

base_job_name = 'vicuna-demo'         

instance_type = 'ml.p4d.24xlarge'

estimator = Estimator(role=role,
                      entry_point='ds-train-distribute.sh',
                      source_dir='./FastChat/',
                      base_job_name=base_job_name,
                      instance_count=2,
                      instance_type=instance_type,
                      image_uri=image_uri,
                      environment=environment,
                      disable_profiler=True,
                      debugger_hook_config=False,
                      max_run=24*60*60*2)

estimator.fit()
#estimator.fit(inputs)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


Using provided s3_resource


INFO:sagemaker:Creating training-job with name: vicuna-demo-2023-07-21-08-56-32-112


2023-07-21 08:56:36 Starting - Starting the training job......
2023-07-21 08:57:12 Starting - Preparing the instances for training.....................
2023-07-21 09:01:03 Downloading - Downloading input data...
2023-07-21 09:01:18 Training - Downloading the training image........................
2023-07-21 09:05:14 Training - Training image download completed. Training in progress.....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-07-21 09:06:08,802 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-07-21 09:06:08,862 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-07-21 09:06:08,871 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-07-21 09:06:08,873 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-07-21 09:06:10,092 sagemaker-training-toolkit INFO  

You could find the model path in S3 from above logs.

In [104]:
!aws s3 ls s3://sagemaker-us-west-2-687912291502/llama/output/2023-07-19-15-07-44/llama_out/

2023-07-19 15:07:45         21 added_tokens.json
2023-07-19 15:07:45        545 config.json
2023-07-19 15:07:45        132 generation_config.json
2023-07-19 15:07:45 13476958625 pytorch_model.bin
2023-07-19 15:07:45        423 special_tokens_map.json
2023-07-19 15:07:45     499723 tokenizer.model
2023-07-19 15:07:45        736 tokenizer_config.json
2023-07-19 15:07:45       4795 training_args.bin
